In [22]:
#imports
import string
import pandas as pd
import pickle as pkl
from collections import Counter
from matplotlib import pyplot as plt

import numpy as np
from random import sample

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import sys
sys.path.append('../../src')
import utils

In [23]:
videos = pd.read_pickle('../../data/embeddings/openai-title')

In [24]:
embs = [videos[v] for v in videos]

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(embs, embs, test_size=0.2)

In [26]:
class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
          layers.Dense(1536, activation='relu'),
          layers.Dense(768, activation='relu'),
          layers.Dense(384, activation='relu'),
          layers.Dense(latent_dim, activation='tanh'),
        ])
        
        self.decoder = tf.keras.Sequential([
          layers.Dense(384, activation='relu'),
          layers.Dense(768, activation='relu'),
          layers.Dense(1536, activation='tanh')
          # layers.Dense(1459, activation='relu'),
          # layers.Dense(14592, activation='tanh')
          # layers.Reshape((-1,768))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(128)

autoencoder.compile(optimizer='adam', loss='cosine_similarity', metrics='cosine_similarity')

In [27]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [28]:
autoencoder.fit(X_train, X_train, epochs=10, shuffle=True, batch_size=64, validation_data=(X_test, X_test), verbose=True)

Epoch 1/10
6485/6485 [==============================] - 232s 36ms/step - loss: -0.9141 - cosine_similarity: 0.9141 - val_loss: -0.9240 - val_cosine_similarity: 0.9240
Epoch 2/10
6485/6485 [==============================] - 226s 35ms/step - loss: -0.9271 - cosine_similarity: 0.9271 - val_loss: -0.9271 - val_cosine_similarity: 0.9271
Epoch 3/10
6485/6485 [==============================] - 237s 37ms/step - loss: -0.9298 - cosine_similarity: 0.9298 - val_loss: -0.9296 - val_cosine_similarity: 0.9296
Epoch 4/10
6485/6485 [==============================] - 225s 35ms/step - loss: -0.9312 - cosine_similarity: 0.9312 - val_loss: -0.9307 - val_cosine_similarity: 0.9307
Epoch 5/10
6485/6485 [==============================] - 225s 35ms/step - loss: -0.9321 - cosine_similarity: 0.9321 - val_loss: -0.9315 - val_cosine_similarity: 0.9315
Epoch 6/10
6485/6485 [==============================] - 224s 35ms/step - loss: -0.9327 - cosine_similarity: 0.9327 - val_loss: -0.9319 - val_cosine_similarity: 0.931

KeyboardInterrupt: 

In [30]:
autoencoder.save_weights('../../data/models/autoencoder-titles')

In [6]:
autoencoder = autoencoder.load_weights('../../data/models/autoencoder-titles')

In [29]:
autoencoder.evaluate(X_test, X_test)

3243/3243 [==============================] - 30s 9ms/step - loss: -0.9331 - cosine_similarity: 0.9331


[-0.9330801963806152, 0.9330801963806152]

In [35]:
embeddings = dict()

for video in tqdm(videos):
    emb = videos[video]
    emb = np.array([emb])
    emb = autoencoder.encoder(emb)
    embeddings[video] = emb

100%|██████████| 518735/518735 [49:56<00:00, 173.10it/s]


In [36]:
pkl.dump(embeddings, open('../../data/embeddings/title-autoencoded', 'wb'))